# Contexte du projet
Au sein du bureau d'études, on vous indique une source de données sur les Chiffres-clés de l'épidémie de COVID19, dont on souhaite obtenir des données de synthèse.

On vous demande d'effectuer le traitement de ces données, suivant certains critères, puis de fournir un résultat de ce traitement.

---

# Modalités pédagogiques

On souhaite obtenir des informations de synthèse d’après des données sur les Chiffres-clés de l'épidémie de COVID19.

Récupérer les données diffusées en « open data » sur le site https://www.data.gouv.fr/fr/ . Chercher le jeu de données sur les Chiffres-clés concernant l'épidémie de COVID19 et télécharger les données au format CSV.

On s’intéresse au nombre d’hospitalisations.

Ecrire un programme en python qui permette de connaitre le nombre d’hospitalisations cumulé suivant le département ou la région choisie, et sur la période choisie.

Le programme demande à l’utilisateur s’il veut connaitre les résultats pour un département ou une région (D ou R), puis demande le numéro de la région ou du département, puis demande une date de départ (au format 0420 par exemple pour avril 2020), et enfin une date de fin.

A l’issue du traitement, le programme affiche le nombre d’hospitalisation cumulée du lieu considéré et sur la période considérée.

In [ ]:
#---Fonction qui gère l'ouverture et le stockage du fichier CSV---
def ouverture():
    with open ("chiffres-cles.csv", "r", encoding = 'utf-8') as file: # On ouvre le fichier avec l'encodage UTF-8
        fichier = file.read().split('\n') # On recupère les données en string qu'on transforme en liste de chaque ligne

    fichier.pop(0) # On supprime les noms des colones

    liste_covid = [ligne.split(',') for ligne in fichier] # On sépare chaque ligne en une liste de ces éléments, notre liste final à deux dimensions
    
    return (liste_covid) # On revoie la liste obtenu

#---Fonction qui calcule le nombre d'hospitalisations pour les paramètres donnés---
def recherche(zone, num, depart, fin):
    covid = ouverture() # On récupère les données grâce à notre fonction 'ouverture'
    nombre_hospi = 0 # Initialisation de nombre cumulé d'hospitalisations
    nom_zone = '' # Initialisation du nom de la zone

    for ligne in covid: # Pour chaque ligne (format liste) de notre CSV

        date = ligne[0].split('-') # On transforme la date en une liste [année, mois, jour]
        date = [int(chiffre) for chiffre in date] # On transforme les chiffres entrées en 'int' pour comparaison
        code_zone = ligne[2].split('-') # On transforme le code de la zone en une liste [Type(region ou departement), numéro]

        zone_match = code_zone[0][0] == zone and code_zone[1] == num # Booléen qui teste si la zone de la ligne correspond à la zone recherché
        date_inf_match = depart[0] <= date[0] and (depart[1] <= date[1] or depart[0] < date[0]) # Booléen qui teste si la date de départ entrée est bien inférieur ou égal à la date de la ligne
        date_sup_match = fin[0] >= date[0] and (fin[1] >= date[1] or fin[0] > date[0]) # Booléen qui teste si la date de fin est supérieur ou égale à la date de la ligne

        if ligne[12] and zone_match and date_inf_match and date_sup_match: # Si toutes nos conditions sont validées et que le nombre d'hospitalisations pour cette ligne existe
            print(ligne)
            nombre_hospi += int(ligne[12]) # On ajoute le nombre d'hospitalisations, convertit en 'int', de cette ligne à notre nombre total
            nom_zone = ligne[3] # On récupère le nom de la zone
    
    return (nombre_hospi, nom_zone) # On renvoit un tuple (nombre total d'hospitalisations pour ces conditions, le nom entier du département/région pour affichage)

#---Fonction main pour la saisie et l'affichage---   
def main():
    # Boucle principale
    while True:
        
        #Boucle de saisie du type de zone
        while True:
            zone = input("Souhaitez vous connaitre les hospitalisations d'un département (D) ou d'une région (R) ?)\n").upper() # Saisie convertit en majuscule

            if zone == 'R' or zone == 'D': # Si zone valide, on sort de la boucle
                break

            else:
                print(f"\n'{zone}' n'est pas une saisie valide\n") # Sinon on recommence
        

        #Saisie du numéro de zone
        zone_print = "département" if zone == 'D' else "région" # Transformation de la zone en texte pour l'affichage
        numero = input(f"Quel est votre numéro de {zone_print} ?\n") #On récupère le numéro de zone
                

        #Boucle de saisie des dates
        while True:
            depart_entree = input("Sélectionnez une date de départ (mois/année).\n") # On récupère la date d'où l'on souhaite partir
            depart_entree = depart_entree.split('/')[::-1] # On transforme la date en une liste [mois, année] et on inverse l'ordre pour avoir le même que la date du CSV

            fin_entree = input("Sélectionnez une date de fin (mois/année).\n") # Comme précédemment mais pour la date d'où l'on souhaite arriver
            fin_entree = fin_entree.split('/')[::-1]

            # On convertit les dates de départ et d'arrivée en 'int' pour comparaison. Si impossible, on recommence la saisie
            try:
                depart = [int(chiffre) for chiffre in depart_entree]
                fin = [int(chiffre) for chiffre in fin_entree]

            except:
                print("Saisies de dates invalides, veuillez recommencer\n")
            
            
            if depart[0] > fin[0] or (depart[1] > fin[1] and depart[0] == fin[0]): # Si la date de départ est supérieur à la date d'arrivée
                print("Saisies de dates invalides, veuillez recommencer\n") # On affiche une érreur et on recommence la saisie
            
            else: # Sinon on sort de la boucle et continue le programme
                break


        #Affichage du résultat
        resultat = recherche(zone, numero, depart, fin) # On récupère le tuple (nombre d'hospitalisations, nom de la zone) après appelle à la fonction 'recherche'

        if resultat[1]: # Si on a bien trouvé la zone cherché
            print(f"Nombres d'hospitalisations en {resultat[1]} du {depart[1]}/{depart[0]} au {fin[1]}/{fin[0]}: {resultat[0]}.\n") # Affichage du résultat 

        else: # Si on n'a pas trouvé la zone cherché ou rien entre ces dates
            print(f"{zone_print} n°{numero} introuvable ou pas d'hospitalisations pour cette date")


        #On demande si on veut recommencer ou arrêter là
        continuer = input("Voulez vous recommencer (yes/no)?")

        if continuer != 'yes':
            break

main()